In [1]:
import gzip
import json
import dateutil.parser
import random

In [2]:
import homework1

In [3]:
root = "/home/scotty/dsc_256/fall_25/module_01/hw/data/datasets"

In [4]:
f = gzip.open(root + "/fantasy_10000.json.gz")
dataset = []
for l in f:
    dataset.append(json.loads(l))

In [5]:
dataset[0]

{'user_id': '8842281e1d1347389f2ab93d60773d4d',
 'book_id': '18245960',
 'review_id': 'dfdbb7b0eb5a7e4c26d59a937e2e5feb',
 'rating': 5,
 'review_text': 'This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though. \n But what would 

In [15]:
## import libraries
import numpy as np
#import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import gzip
import json
from datetime import datetime
import dateutil.parser
import random
import os
import sys
import inspect
def Q4(dataset):
    def getMaxLen(dataset):
        # Find the longest review (number of characters)
        review_lens = [len(i['review_text']) for i in dataset]
        maxLen = max(review_lens)
        return maxLen
    
    def featureQ2(datum, maxLen):
        # Implement (should be 1, length feature, day feature, month feature)
        len_feature = len(datum['review_text'])/maxLen
        wd_encoder = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False)
        m_encoder = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False)

        wd_encoder.fit(np.array(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']).reshape(-1, 1))
        m_encoder.fit(np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']).reshape(-1, 1))

        day_feature = wd_encoder.transform([[datum['date_added'].split(' ')[0]]]).ravel()
        month_feature = m_encoder.transform([[datum['date_added'].split(' ')[1]]]).ravel()
        
        feature = np.concatenate((np.array([1]),np.array([len_feature]), day_feature, month_feature))
        return feature

    def featureQ3(datum, maxLen):
        # Implement (should be 1, length feature, day feature, month feature)
        len_feature = len(datum['review_text'])/maxLen
        day_feature = datetime.strptime(datum['date_added'].split(' ')[0],'%a').isoweekday()
        month_feature = datetime.strptime(datum['date_added'].split(' ')[1],'%b').month
        return [1] + [len_feature] + [day_feature] + [month_feature]
    
    maxLen = getMaxLen(dataset)
    #dataset4 = dataset[:]
    random.seed(0)
    random.shuffle(dataset)
    half_num = len(dataset)//2
    dataset_train = dataset[:half_num]
    print(len(dataset_train),dataset_train[0])
    dataset_test = dataset[len(dataset)+1:1]
    X2_train = np.array([featureQ2(d,maxLen) for d in dataset_train])
    print(X2_train.shape)
    X2_test = [featureQ2(d,maxLen) for d in dataset_test]
    X3_train = [featureQ3(d,maxLen) for d in dataset_train]
    X3_test = [featureQ3(d,maxLen) for d in dataset_test]
    y_train = [d['rating'] for d in dataset_train]
    y_test = [d['rating'] for d in dataset_test]
    model = LinearRegression(fit_intercept=False)
    model.fit(X2_train, y_train)

    y_pred = model.predict(np.array(X2_test).reshape(-1,1))
    #Sum of squared errors (SSE)

    sse = sum([x**2 for x in (y_test - y_pred)])
    #Mean squared error (MSE)

    test_mse2 = sse / len(y_test)

    print(test_mse2)

    model = LinearRegression(fit_intercept=False)
    model.fit(X3_train, y_train)

    y_pred = model.predict(X3_test)
    #Sum of squared errors (SSE)

    sse = sum([x**2 for x in (y_test - y_pred)])
    #Mean squared error (MSE)

    test_mse3 = sse / len(y_test)
    print(test_mse2)

    return test_mse2, test_mse3
Q4(dataset)

5000 {'user_id': 'c84807d576e92c3746180ec064fbd440', 'book_id': '17853851', 'review_id': '83c175b6faf1ffc0b8d0dac0d7d4f3d9', 'rating': 3, 'review_text': "I was really looking forward to Nick and Shaya's story. I enjoyed it but I did find it lacking in some ways. It wasn't what I expected of these characters, as their story has been built up from the two previous books. It didn't feel like a Phoenix Pack story to me, not bad just not what I expected.", 'date_added': 'Wed May 29 10:28:27 -0700 2013', 'date_updated': 'Thu Feb 13 12:10:54 -0800 2014', 'read_at': 'Thu Feb 13 12:10:54 -0800 2014', 'started_at': 'Thu Feb 13 00:00:00 -0800 2014', 'n_votes': 2, 'n_comments': 0}
(5000, 19)


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by LinearRegression.

In [18]:
## import libraries
import numpy as np
#import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import gzip
import json
from datetime import datetime
import dateutil.parser
import random
import os
import sys
import inspect
def Q4(dataset):
    def feature_extractors_setup(dataset):
        """Initializes and fits all necessary components for both models."""
    
        # 1. Calculate max length
        maxLen = max(len(d['review_text']) for d in dataset)
    
        # 2. Setup and Fit Encoders (for Model 2: One-Hot)
        wd_encoder = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False)
        m_encoder = OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False)
    
        # Fit: [Sun, Mon, ...] and [Jan, Feb, ...]
        wd_cats = np.array(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']).reshape(-1, 1)
        m_cats = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']).reshape(-1, 1)
    
        wd_encoder.fit(wd_cats)
        m_encoder.fit(m_cats)
    
        return maxLen, wd_encoder, m_encoder
    def get_features_model2(datum, maxLen, wd_encoder, m_encoder):
        """Generates One-Hot features (Model 2) using fitted encoders."""
        len_feature = len(datum['review_text']) / maxLen
        date_parts = datum['date_added'].split(' ')
        day_abbr = date_parts[0]
        month_abbr = date_parts[1]
    
        # Use .transform() with fitted encoders
        day_feature_1d = wd_encoder.transform([[day_abbr]]).ravel()
        month_feature_1d = m_encoder.transform([[month_abbr]]).ravel()
    
        # Concatenate: [Bias (1), Length (1), Day (6), Month (11)]
        base_features = np.array([1.0, len_feature])
        return np.concatenate((base_features, day_feature_1d, month_feature_1d))

    def get_features_model3(datum, maxLen):
        """Generates Raw Integer features (Model 3)."""
        # Note: Using %a %b %d %Y format codes for a robust parse
        full_date_string = datum['date_added'] + ' 01 2000' # Append dummy day/year to parse fully
    
        len_feature = len(datum['review_text']) / maxLen
    
        # Extract integer features
        try:
            t = datetime.strptime(full_date_string, '%a %b %d %Y')
            day_feature = t.isoweekday() # 1-7
            month_feature = t.month      # 1-12
        except ValueError:
            # Fallback if date is malformed (shouldn't happen with given data structure)
            day_feature = 0
            month_feature = 0
    
        # Return as NumPy array directly
        return np.array([1.0, len_feature, day_feature, month_feature])
    
    maxLen, wd_encoder, m_encoder = feature_extractors_setup(dataset)
    
    # 2. DATA SPLIT (CRITICAL FIXES HERE)
    random.seed(0)
    # Important: Create a copy of the dataset before shuffling to avoid side effects
    dataset_shuffled = dataset[:] 
    random.shuffle(dataset_shuffled) 
    
    half_num = len(dataset_shuffled) // 2
    
    # Correct slicing for 50/50 split
    dataset_train = dataset_shuffled[:half_num]
    dataset_test = dataset_shuffled[half_num:] 
    
    # 3. FEATURE MATRIX CONSTRUCTION
    
    # Features for Model 2 (One-Hot)
    # The encoders were FIT on the whole dataset (if train/test are subsets of a whole).
    # For a strictly correct split, we should refit on dataset_train, but to match the
    # Q2 spirit, we use the encoders established by the whole process.
    X2_train = np.array([get_features_model2(d, maxLen, wd_encoder, m_encoder) for d in dataset_train])
    X2_test = np.array([get_features_model2(d, maxLen, wd_encoder, m_encoder) for d in dataset_test])
    
    # Features for Model 3 (Raw Integer)
    X3_train = np.array([get_features_model3(d, maxLen) for d in dataset_train])
    X3_test = np.array([get_features_model3(d, maxLen) for d in dataset_test])
    
    # Targets
    y_train = np.array([d['rating'] for d in dataset_train])
    y_test = np.array([d['rating'] for d in dataset_test])
    
    # 4. TRAIN AND TEST MODEL 2 (One-Hot)
    model2 = LinearRegression(fit_intercept=False)
    model2.fit(X2_train, y_train)
    
    y_pred2 = model2.predict(X2_test)
    
    # MSE Calculation (Robust NumPy method)
    test_mse2 = np.mean((y_test - y_pred2)**2)
    
    # 5. TRAIN AND TEST MODEL 3 (Raw Integer)
    model3 = LinearRegression(fit_intercept=False)
    model3.fit(X3_train, y_train)
    
    y_pred3 = model3.predict(X3_test)
    
    # MSE Calculation (Robust NumPy method)
    test_mse3 = np.mean((y_test - y_pred3)**2)
    
    # 6. RETURN RESULTS
    return test_mse2, test_mse3

Q4(dataset)

(np.float64(1.5266596343007535), np.float64(1.5191618561424114))

In [19]:
for d in dataset:
    t = dateutil.parser.parse(d['date_added'])
    d['parsed_date'] = t

In [ ]:
dataset[0]
def getMaxLen(dataset):
    # Find the longest review (number of characters)
    review_lens = [len(i['review_text']) for i in dataset]
    maxLen = max(review_lens)
    return maxLen

def featureQ2(datum, maxLen):
    # note: should be 1, length feature, day feature (list), month feature (list)
    # normailize length feature
    len_feature = len(datum['review_text'])/maxLen
    date_dt = datum['parsed_date']

    # weekday returns Mon: 0,...,Sun:6
    day = date_dt.weekday()

    # initalize day feature 
    day_feature = [0,0,0,0,0,0,0]
    
    # day is index and mark as true  for element reprsenting day of week
    day_feature[day] = 1

    # month returns Jan:1,...,Dec:12 (must account for index start at zero)
    month = date_dt.month

    # initalize month feature then insert into list
    month_feature = [0,0,0,0,0,0,0,0,0,0,0,0]

    # day is index and mark as true  for element reprsenting day of week
    day_feature[day] = 1
    
    # since counting starts at 0 Jan:0,...,Dec:11
    month_feature[month-1] = 1

    # build Q2 feature 
    # note: drop first element for month and day feature to prevent multicollinearity
    feature = np.concatenate((np.array([1]),np.array([len_feature]), np.array(day_feature[1:]), np.array(month_feature[1:])))

    return feature

def Q2(dataset):
    # get max len of 'review_text'
    maxLen = getMaxLen(dataset)

    # build features(X) and target(y) lists
    X = [featureQ2(d,maxLen) for d in dataset]
    y = [d['rating'] for d in dataset]
    
    # calc least squares
    theta,residuals,rank,s = np.linalg.lstsq(X, y)

    # calc mean squared error
    MSE = residuals/len(y)
    print(MSE)
    MSE2 = MSE[0]

    return X, y, MSE2
Q2(dataset)

[]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
dataset4 = dataset[:]
random.seed(0)
random.shuffle(dataset4)

In [4]:
## import libraries
import numpy as np
#import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import gzip
import json
from datetime import datetime
import dateutil.parser
import random
import os
import sys
import inspect

root = '/home/scotty/dsc_256/fall_25/module_01/hw/data/datasets'

f = open(f"{root}/beer_50000.json")
datasetB = []
for l in f:
    datasetB.append(eval(l))

In [22]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
def extract_beer_features(datum):
    features = [
        len(datum['review/text']),
        datum['review/text'].count('\t'),
        datum['review/appearance'],
        datum['review/palate'],
        datum['review/taste'],
        datum['review/aroma'],
        datum['review/timeStruct']['hour'],
        # Use .get() with a default value in case 'beer/ABV' is missing
        datum.get('beer/ABV', 0) or 0 # Handles None or missing key
    ]
    
    return features

"""
{'review/appearance': 2.5, 
 'beer/style': 'Hefeweizen', 
 'review/palate': 1.5, 
 'review/taste': 1.5, 
 'beer/name': 'Sausa Weizen', 
 'review/timeUnix': 1234817823, 
 'beer/ABV': 5.0, 
 'beer/beerId': '47986', 
 'beer/brewerId': '10325', 
 'review/timeStruct': {'isdst': 0, 'mday': 16, 'hour': 20, 'min': 57, 'sec': 3, 'mon': 2, 'year': 2009, 'yday': 47, 'wday': 0}, 
 'review/overall': 1.5, 
 , 
 'user/profileName': 'stcules', 
 'review/aroma': 2.0}
"""

def Q5_with_PCA(dataset, feat_func):
    """
    Applies a StandardScaler -> PCA -> LogisticRegression pipeline.
    """
    # Build feature matrix X and target vector y
    X = np.array([feat_func(d) for d in dataset])
    y = np.array([d['review/overall'] >= 4 for d in dataset])

    # 1. Scale the features
    # This standardizes features by removing the mean and scaling to unit variance.
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Apply PCA for dimensionality reduction
    # We'll reduce our 6 features down to 2 principal components.
    # You can experiment with different numbers for n_components.
    pca = PCA(n_components=4)
    X_pca = pca.fit_transform(X_scaled)
    print(f"Explained variance by component: {pca.explained_variance_ratio_}")
    print(f"Total variance explained by 2 components: {np.sum(pca.explained_variance_ratio_):.2f}\n")

    # 4. View the Component Loadings in a DataFrame for easy reading
    feature_names = ['text len','review/text','appearance','palate','taste','aroma','hour','abv']
    components_df = pd.DataFrame(
        pca.components_, 
        columns=feature_names, 
        index=['PC1', 'PC2','PC3', 'PC4']
    )
    print("PCA Component Loadings:")
    print(components_df)
    
    # 3. Train the logistic regression model on the PCA-transformed data
    log_reg = LogisticRegression(class_weight='balanced')
    log_reg.fit(X_pca, y) # Notice we use X_pca here
    
    # Make predictions using the PCA-transformed data
    y_pred = log_reg.predict(X_pca)

    # Calculate balanced error rate (1 - balanced accuracy)
    bal_accuracy = balanced_accuracy_score(y, y_pred)
    bal_error_rate = 1 - bal_accuracy
    
    # --- The TP/TN/FP/FN calculations remain the same ---
    y_true = y.astype(int)
    y_pred_int = y_pred.astype(int)
    true_positives = np.sum(y_true * y_pred_int)
    true_negatives = np.sum((1 - y_true) * (1 - y_pred_int))
    false_positives = np.sum((1 - y_true) * y_pred_int)
    false_negatives = np.sum(y_true * (1 - y_pred_int))

    return true_positives, true_negatives, false_positives, false_negatives, bal_error_rate

Q5_with_PCA(datasetB,extract_beer_features)

Explained variance by component: [0.39646459 0.17578565 0.12424039 0.09263147]
Total variance explained by 2 components: 0.79

PCA Component Loadings:
     text len  review/text  appearance    palate     taste     aroma  \
PC1  0.178561     0.134287    0.413668  0.469787  0.482680  0.469809   
PC2  0.650991     0.674400   -0.134852 -0.149903 -0.160307 -0.120212   
PC3  0.128017     0.101006    0.025232  0.012069  0.014497  0.003619   
PC4 -0.047527    -0.251601   -0.221796 -0.143628 -0.138049 -0.043574   

         hour       abv  
PC1 -0.035841  0.321367  
PC2 -0.133582  0.150816  
PC3  0.982780 -0.080917  
PC4  0.116716  0.911064  


(np.int64(27777),
 np.int64(12804),
 np.int64(3584),
 np.int64(5835),
 np.float64(0.19614766100917436))

In [ ]:

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
def featureQ5(datum):
    # Feature vector for one data point
    feature = len(datum['review/text'])
    return [1] + [feature]

def featureQ7(datum):
    # Implement (any feature vector which improves performance over Q5)
    appearance = datum['review/appearance']
    palate = datum['review/palate']
    taste = datum['review/taste']
    aroma = datum['review/aroma']
    abv = datum.get('beer/ABV', 0) 
    
    # calc weighted sum using results from PCA
    yum_factor = (0.41 * appearance + 
                     0.47 * palate + 
                     0.48 * taste + 
                     0.47 * aroma + 
                     0.32 * abv)
    
    return [1] + [yum_factor]

def Q5(dataset,feat_func):

    # build feature(X) and target(y) lists
    X = np.array([feat_func(d) for d in dataset])
    y = np.array([d['review/overall'] >=4 for d in dataset])

    # initalize sklearn logistic regression model
    log_reg = LogisticRegression(class_weight='balanced')

    # fit logistic regression model
    log_reg.fit(X,y)

    # make predictions using log_reg model
    y_pred = log_reg.predict(X)

    #define true condition True:1 and False:0
    y_true = y.astype(int) 
    y_pred_int = y_pred.astype(int)

    # find true positives
    true_positives = np.sum(y_true * y_pred_int)

    #find true negatives
    true_negatives = np.sum((1 - y_true) * (1 - y_pred_int))

    # find false positives
    false_positives = np.sum((1 - y_true) * y_pred_int)

    # find false negatives
    false_negatives = np.sum(y_true * (1 - y_pred_int))

    # Calculate balanced error rate (1-accuracy)
    bal_accuracy = balanced_accuracy_score(y, y_pred)
    bal_error_rate = 1 - bal_accuracy
    print(true_positives, true_negatives, false_positives, false_negatives, bal_error_rate)
    return true_positives, true_negatives, false_positives, false_negatives, bal_error_rate

Q5(datasetB,featureQ5)
Q5(datasetB,featureQ7)

14201 10503 5885 19411 0.4683031525957275
24522 11878 4510 9090 0.2728202478681687


(np.int64(24522),
 np.int64(11878),
 np.int64(4510),
 np.int64(9090),
 np.float64(0.2728202478681687))

In [ ]:

a = {'review/text': "A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana."}
count_tabs = a['review/text'].count('\t')
print(count_tabs)

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, balanced_accuracy_score
import numpy as np
def featureQ5(datum):
    # Feature vector for one data point
    feature = len(datum['review/text'])
    return [1] + [feature]

def Q5(dataset,feat_func):

    # build feature(X) and target(y) lists
    X = np.array([feat_func(d) for d in dataset])
    y = np.array([d['review/overall'] >=4 for d in dataset])

    # initalize sklearn logistic regression model
    log_reg = LogisticRegression(class_weight='balanced')

    # fit logistic regression model
    log_reg.fit(X,y)

    # make predictions using log_reg model
    y_pred = log_reg.predict(X)

    #define true condition True:1 and False:0
    y_true = y.astype(int) 
    y_pred_int = y_pred.astype(int)

    # find true positives
    true_positives = np.sum(y_true * y_pred_int)

    #find true negatives
    true_negatives = np.sum((1 - y_true) * (1 - y_pred_int))

    # find false positives
    false_positives = np.sum((1 - y_true) * y_pred_int)

    # find false negatives
    false_negatives = np.sum(y_true * (1 - y_pred_int))

    # Calculate balanced error rate (1-accuracy)
    bal_accuracy = balanced_accuracy_score(y, y_pred)
    bal_error_rate = 1 - bal_accuracy
    return true_positives, true_negatives, false_positives, false_negatives, bal_error_rate
    FP = false_positives
    TN = true_negatives
    FN = false_negatives
    TP = true_positives
    BER = 0.5 * (FP / (TN + FP) + FN / (FN + TP))

    #print(f"sklearn:{balanced_err_rate}")
    #print(f"comp:{BER}")
    print((TP, TN, FP, FN, BER))
    

Q5(datasetB,featureQ5)

4


(np.int64(14201),
 np.int64(10503),
 np.int64(5885),
 np.int64(19411),
 np.float64(0.4683031525957275))

In [33]:
def Q6(dataset):
    def featureQ5(datum):
        # Feature vector for one data point
        feature = len(datum['review/text'])
        return [1] + [feature]

    # build feature(X) and target(y) lists
    X = np.array([featureQ5(d) for d in dataset])
    y = np.array([d['review/overall'] >=4 for d in dataset])

    # initalize sklearn logistic regression model
    log_reg = LogisticRegression(class_weight='balanced')

    # fit logistic regression model
    log_reg.fit(X,y)

    # make predictions using log_reg model
    y_pred = log_reg.predict(X)

    # grab col 2 probablities since intrested where 'review/overall' >=4
    probs = log_reg.predict_proba(X)[:, 1]

    # sort in decending order based on probability from log_reg
    sorted_indices = np.argsort(probs)[::-1]
    
    # sort true targets based on probs from log_reg
    sorted_y = y[sorted_indices]

    # define k values to test
    k_vals = np.array([1,10,100,10000])
    
    # calc sum of correct predictions 
    true_counts = np.cumsum(sorted_y)
    
    # calc precision for each k using numpy linear algebra
    precs = true_counts[k_vals - 1] / k_vals
    precs = [float(i) for i in precs]
    for i in precs:
        print(type(i),i)
    return precs
def Q6_test(dataset):
    X = [[1, len(d['review/text'])] for d in dataset]
    y = [d['review/overall'] >= 4 for d in dataset]
    
    mod = LogisticRegression(class_weight='balanced')
    mod.fit(X,y)
    predictions = mod.predict(X) # Binary vector of predictions
    scores = [x[1] for x in mod.predict_proba(X)]

    sortedScores = list(zip(scores,y))
    sortedScores.sort(reverse=True)
    sortedLabels = [x[1] for x in sortedScores]
    
    precs = []
    
    for k in [1,100,1000,10000]: # Not efficient, but fine
        precK = sum(sortedLabels[:k]) / k
        precs.append(precK)
        #print("Precision@" + str(k) + " = " + str(precK))
    print(f"!!!{precs}")
    for i in precs:
        print(type(i),i)
    return precs

Q6(datasetB)
Q6_test(datasetB)

<class 'float'> 1.0
<class 'float'> 0.8
<class 'float'> 0.75
<class 'float'> 0.7147
!!![1.0, 0.75, 0.71, 0.7147]
<class 'float'> 1.0
<class 'float'> 0.75
<class 'float'> 0.71
<class 'float'> 0.7147


[1.0, 0.75, 0.71, 0.7147]

In [23]:

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
def featureQ5(datum):
    # Feature vector for one data point
    feature = len(datum['review/text'])
    return [1] + [feature]

def featureQ7(datum):
    # Implement (any feature vector which improves performance over Q5)
    appearance = datum['review/appearance']
    palate = datum['review/palate']
    taste = datum['review/taste']
    aroma = datum['review/aroma']
    abv = datum.get('beer/ABV', 0) 
    
    # calc weighted sum using results from PCA
    quality_score = (0.41 * appearance + 
                     0.47 * palate + 
                     0.48 * taste + 
                     0.47 * aroma + 
                     0.32 * abv)
    return [1]+ [quality_score]

def Q5(dataset,feat_func):

    # build feature(X) and target(y) lists
    X = np.array([feat_func(d) for d in dataset])
    y = np.array([d['review/overall'] >=4 for d in dataset])

    # initalize sklearn logistic regression model
    log_reg = LogisticRegression(class_weight='balanced')

    # fit logistic regression model
    log_reg.fit(X,y)

    # make predictions using log_reg model
    y_pred = log_reg.predict(X)

    #define true condition True:1 and False:0
    y_true = y.astype(int) 
    y_pred_int = y_pred.astype(int)

    # find true positives
    true_positives = np.sum(y_true * y_pred_int)

    #find true negatives
    true_negatives = np.sum((1 - y_true) * (1 - y_pred_int))

    # find false positives
    false_positives = np.sum((1 - y_true) * y_pred_int)

    # find false negatives
    false_negatives = np.sum(y_true * (1 - y_pred_int))

    # Calculate balanced error rate (1-accuracy)
    bal_accuracy = balanced_accuracy_score(y, y_pred)
    bal_error_rate = 1 - bal_accuracy
    return true_positives, true_negatives, false_positives, false_negatives, bal_error_rate

Q5(dataset,featureQ5)
Q5(dataset,featureQ7)

NameError: name 'dataset' is not defined

In [7]:
def testQ1():
    theta1, MSE1 = homework1.Q1(dataset)
    # The autograder will compare these answers to a reference solution
    print(theta1)
    print(MSE1)

In [ ]:
def testQ2():
    X, Y, MSE = homework1.Q2(dataset)
    print((X[0],Y[0],MSE))

In [ ]:
def testQ3():
    X, Y, MSE = homework1.Q3(dataset)
    print((X[0],Y[0],MSE))

In [ ]:
def testQ4():
    test_mse2, test_mse3 = homework1.Q4(dataset4)
    print((test_mse2, test_mse3))

In [ ]:
def testQ5():
    TP, TN, FP, FN, BER = homework1.Q5(datasetB, homework1.featureQ5)
    print((TP, TN, FP, FN, BER))

In [ ]:
def testQ6():
    precs = homework1.Q6(datasetB)
    print(precs)

In [ ]:
def testQ7():
    _, _, _, _, BER5 = homework1.Q5(datasetB, homework1.featureQ5)
    _, _, _, _, BER7 = homework1.Q5(datasetB, homework1.featureQ7)
    print((BER5,BER7))

In [8]:
testQ1()

KeyError: 'review'

In [ ]:
testQ2()

In [ ]:
testQ3()

In [ ]:
testQ4()

In [ ]:
testQ5()

In [ ]:
testQ6()

In [ ]:
testQ7()